In [ ]:
import os
import numpy as np

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt 
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from shapely.geometry import MultiPolygon, Polygon
import rasterio as rio
from rasterio.plot import show
import rasterio.mask

import zipfile



### Convert AOI's
Extract golfo dulce and bahia colebra area of interest from the shape file and convert them to geojson to use with the [sentinelsat api](https://sentinelsat.readthedocs.io/en/stable/api.html).



In [ ]:
aoi = gpd.read_file('RCCR_working_areas/RCCR_working_areas_poly_fix.shp')
aoi.info()

In [ ]:
golfo_dulce = aoi.drop([aoi.index[0]])
golfo_dulce.to_file("RCCR_working_areas/golfo_dulce_aoi.json", driver="GeoJSON")

In [ ]:
golfo_dulce['geometry'].plot()


In [ ]:
bahia_colebra = aoi.drop([aoi.index[1]])
bahia_colebra.to_file("RCCR_working_areas/bahia_colebra.json", driver="GeoJSON")
bahia_colebra['geometry'].plot()

### Instantiate sentinel api
Replace user and password with own credential used to login with [copernicus scihub](https://scihub.copernicus.eu/dhus)

In [ ]:
user = 'XXX' 
password = 'XXX' 

api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

### Query products from the golfo dulce area
Finds products starting from 01/01/2018 to 04/01/2020 with processing **Level 2A**.

In [ ]:
footprint = geojson_to_wkt(geojson_obj=read_geojson('RCCR_working_areas/golfo_dulce_aoi.json'))
footprint

In [ ]:
products = api.query(footprint,
                     date = ('20180101', '20200401'),
                     platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0,10))
len(products)


In [ ]:
products_gdf = api.to_geodataframe(products)
products_gdf_sorted = products_gdf.sort_values(['beginposition'], ascending=[False])
products_gdf_sorted.head()

### Dowloading Products
Download first (most recent) 5 products from the golfo dulce area, zip files will be created in the data folder.
This might take a while!

In [ ]:
target_dir = 'data'

for i in range(5):
    image_uuid = products_gdf_sorted['uuid'][i]
    prod_download = api.download(image_uuid, directory_path=target_dir)
    filename = prod_download['title']+ '.zip'
    filepath = os.path.join(target_dir, filename)
    with zipfile.ZipFile(filepath,"r") as zip_ref:
        zip_ref.extractall(target_dir)


### Products with high cloud coverage

In [ ]:
print(products_gdf.columns)
products_sorted_cloud = products_gdf.sort_values(['cloudcoverpercentage'], ascending=[False])
products_sorted_cloud[['title','beginposition','cloudcoverpercentage']].head()

### Query products from the bahia colebra area
Finds products with processing **Level 1C** starting form 01/01/2016 to 04/01/2020

In [ ]:
footprint = geojson_to_wkt(geojson_obj=read_geojson('RCCR_working_areas/bahia_colebra.json'))

products = api.query(footprint,
                     date = ('20160101', '20200401'),
                     platformname = 'Sentinel-2',
                     processinglevel = 'Level-1C',
                     cloudcoverpercentage = (0,10))
len(products)

In [ ]:
products_gdf = api.to_geodataframe(products)
products_gdf_sorted = products_gdf.sort_values(['beginposition'], ascending=[False])
products_gdf_sorted.head()

In [ ]:
target_dir = 'data'

for i in range(5):
    image_uuid = products_gdf_sorted['uuid'][i]
    prod_download = api.download(image_uuid, directory_path=target_dir)
    filename = prod_download['title']+ '.zip'
    filepath = os.path.join(target_dir, filename)
    with zipfile.ZipFile(filepath,"r") as zip_ref:
        zip_ref.extractall(target_dir)

In [ ]:
bahia_colebra_ch = bahia_colebra
bahia_colebra_ch['geometry'] = bahia_colebra['geometry'].convex_hull
bahia_colebra_ch.to_file("RCCR_working_areas/bahia_colebra_ch.json", driver="GeoJSON")
bahia_colebra_ch['geometry'].plot()